In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [2]:
# Load CIFAR-10 dataset and preprocess the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [3]:
# Define the DenseNet model for fine-tuning
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers of the pre-trained model except the last two dense layers
# for layer in base_model.layers:
#     layer.trainable = False

# Compile the model
model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the validation set for monitoring
batch_size = 256
epochs = 20
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), verbose=1)

Epoch 1/20
176/176 [==============================] - 68s 169ms/step - loss: 1.0148 - accuracy: 0.6717 - val_loss: 679.8358 - val_accuracy: 0.1270
Epoch 2/20
176/176 [==============================] - 29s 163ms/step - loss: 0.5515 - accuracy: 0.8161 - val_loss: 1.2875 - val_accuracy: 0.6922
Epoch 3/20
176/176 [==============================] - 27s 156ms/step - loss: 0.3963 - accuracy: 0.8666 - val_loss: 1.2724 - val_accuracy: 0.6898
Epoch 4/20
176/176 [==============================] - 27s 155ms/step - loss: 0.2904 - accuracy: 0.9022 - val_loss: 1.0795 - val_accuracy: 0.7974
Epoch 5/20
176/176 [==============================] - 28s 158ms/step - loss: 0.2257 - accuracy: 0.9230 - val_loss: 1.0359 - val_accuracy: 0.7890
Epoch 6/20
176/176 [==============================] - 28s 159ms/step - loss: 0.1749 - accuracy: 0.9387 - val_loss: 0.9707 - val_accuracy: 0.7770
Epoch 7/20
176/176 [==============================] - 27s 156ms/step - loss: 0.1458 - accuracy: 0.9508 - val_loss: 0.8641 - val_

In [4]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

40/40 [==============================] - 4s 40ms/step - loss: 1.2768 - accuracy: 0.8016
Test Loss: 1.2768080234527588
Test Accuracy: 0.8015999794006348
